In [141]:
time_start = 20201231
time_end =  20210630
today = 20211130

day_back = 5 # 回溯几天进行计算
day = 30 # 一次提取几天的数,一般默认为30
stride = 10 # 一次学习多少天

In [2]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm

ModuleNotFoundError: No module named 'Research'

In [143]:
dataframe_list = DataAPI.get_stock_prices(start_date=20150101, end_date=time_end).reset_index()

In [144]:
dataframe_list = dataframe_list[['timestamp','ticker','pre_close','open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover','chg']]
time_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
time_df['timestamp_shift'] = time_df['timestamp'].apply(lambda x : DataAPI.get_next_trading_day(x, output='datetime'))
dataframe_list = pd.merge(left = dataframe_list , right = time_df , how = 'left',on = 'timestamp')
dataframe_list.drop(['timestamp'],axis=1,inplace=True)
dataframe_list.rename(columns = {"timestamp_shift":"timestamp"},inplace=True)
dataframe_list['timestamp'] = pd.to_datetime(dataframe_list['timestamp']) + pd.Timedelta(hours=9,minutes = 30)
dataframe_list.set_index(['timestamp','ticker'],inplace = True)
dataframe_list

pre_close    open    high     low   close  \
timestamp           ticker                                              
2015-01-06 09:30:00 000001      15.84   15.99   16.28   15.60   16.02   
                    000002      13.90   14.39   15.29   14.22   14.91   
                    000004      15.57   15.58   15.90   15.30   15.69   
                    000005       4.10    4.10    4.10    4.10    4.10   
                    000006       7.05    7.10    7.39    7.02    7.08   
...                               ...     ...     ...     ...     ...   
2021-07-01 09:30:00 688700      46.60   47.80   54.50   47.03   53.38   
                    688777      90.00   89.95   94.80   89.06   94.15   
                    688788     138.20  137.28  148.50  136.33  145.38   
                    688819      43.68   43.40   44.98   43.31   44.49   
                    688981      59.25   59.69   62.50   59.40   61.82   

                                vwap      volume  pct_chg  turnover  \
timestamp           ticker                                            
2015-01-06 09:30:00 000001   15.9605  2860436.43   1.1364    2.9079   
                    000002   14.7858  6560835.70   7.2662    6.7834   
                    000004   15.5709    28095.12   0.7707    3.3496   
                    000005    4.1000        0.00   0.0000    0.0000   
                    000006    7.1881  1078647.12   0.4255    8.0562   
...                              ...         ...      ...       ...   
2021-07-01 09:30:00 688700   50.7153    64336.83  14.5494   19.2222   
                    688777   92.9798    19001.61   4.6111    4.6288   
                    688788  144.3683    13572.56   5.1954    7.7977   
                    688819   44.3727    47249.01   1.8544    4.6754   
                    688981   61.5420   777950.53   4.3376    7.1003   

                            free_turnover   chg  
timestamp           ticker                       
2015-01-06 09:30:00 000001         5.7657  0.18  
                    000002         8.1741  1.01  
                    000004         4.5343  0.12  
                    000005         0.0000  0.00  
                    000006        13.2751  0.03  
...                                   ...   ...  
2021-07-01 09:30:00 688700        19.2222  6.78  
                    688777         4.6288  4.15  
                    688788         7.7977  7.18  
                    688819         4.6754  0.81  
                    688981         7.1003  2.57  

[5357761 rows x 11 columns]

In [160]:
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/ShareFolder/factor_lib")

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [161]:
FT.load_feature_from_dataframe(dataframe_list,universe='Investable')

Loading Feature...: 100%|████████████████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:09<00:00, 172.94it/s]


(                            pre_close    open    high     low   close  \
 timestamp           ticker                                              
 2015-01-16 09:30:00 000001      14.81   14.85   15.35   14.71   15.35   
                     000002      13.10   13.13   13.83   13.00   13.77   
                     000004      16.13   16.00   16.34   15.95   16.10   
                     000005       4.10    4.10    4.10    4.10    4.10   
                     000006       6.38    6.36    6.55    6.35    6.53   
 ...                               ...     ...     ...     ...     ...   
 2021-07-01 09:30:00 688699     237.77  285.32  285.32  285.32  285.32   
                     688777      90.00   89.95   94.80   89.06   94.15   
                     688788     138.20  137.28  148.50  136.33  145.38   
                     688819      43.68   43.40   44.98   43.31   44.49   
                     688981      59.25   59.69   62.50   59.40   61.82   
 
                                 vwa

In [162]:
FT.load_return_data()

Merging feature and return...: 100%|█████████████████████████████████████████████████████████████████████████████| 1580/1580 [00:08<00:00, 190.12it/s]


In [163]:
data_info = FT.data_info

In [170]:
df = pd.DataFrame()
for timestamp in data_info.keys():
    one_data = data_info[timestamp].reset_index()
    one_data['timestamp'] = timestamp
    df = pd.concat([df,one_data],axis=0)
df.sort_values(by = ['timestamp','ticker'],inplace=True)
display(df)

2015-01-06 09:30:00
2015-04-24 09:30:00
2015-01-30 09:30:00
2015-04-03 09:30:00
2015-03-13 09:30:00
2015-01-08 09:30:00
2015-01-07 09:30:00
2015-04-08 09:30:00
2015-01-29 09:30:00
2015-01-12 09:30:00
2015-04-27 09:30:00
2015-04-30 09:30:00
2015-04-28 09:30:00
2015-02-26 09:30:00
2015-02-10 09:30:00
2015-03-04 09:30:00
2015-03-16 09:30:00
2015-03-25 09:30:00
2015-01-22 09:30:00
2015-02-12 09:30:00
2015-04-07 09:30:00
2015-04-20 09:30:00
2015-03-10 09:30:00
2015-02-27 09:30:00
2015-02-04 09:30:00
2015-05-04 09:30:00
2015-04-29 09:30:00
2015-05-05 09:30:00
2015-01-15 09:30:00
2015-04-14 09:30:00
2015-03-30 09:30:00
2015-01-14 09:30:00
2015-04-10 09:30:00
2015-02-06 09:30:00
2015-01-19 09:30:00
2015-05-07 09:30:00
2015-05-06 09:30:00
2015-03-19 09:30:00
2015-03-05 09:30:00
2015-02-02 09:30:00
2015-02-16 09:30:00
2015-05-08 09:30:00
2015-05-11 09:30:00
2015-01-26 09:30:00
2015-01-23 09:30:00
2015-03-03 09:30:00
2015-02-13 09:30:00
2015-03-24 09:30:00
2015-04-16 09:30:00
2015-02-11 09:30:00


2021-06-22 09:30:00
2021-06-23 09:30:00
2021-06-24 09:30:00
2021-06-25 09:30:00
2021-06-29 09:30:00
2021-06-30 09:30:00
2021-07-01 09:30:00
2021-06-28 09:30:00


In [ ]:
dict_output = {}
for timestamp in tqdm(df['timestamp'].dropduplicates()):
    dict_output[timestamp] = {}
for ticker in tqdm(df['ticker'].dropduplicates()):
    one_data = df[df['ticker'] == ticker]
    for i in range(0,array.shape[0] - day ,3): # 其中3 代表取数的步长，ex.每两天取一次数，步长为3
        time_stamp = one_data.iloc[i+day-1]['time_stamp']
        x = one_data.iloc[i:i+day,:]
        y = one_data.iloc[i+day-1]['target']
        dict_output[timestamp][ticker] = x.set_index(['ticker','timestamp'])

In [ ]:
# Save
dictionary = {'hello':'world'}
np.save('my_file.npy', dictionary) 

# Load
read_dictionary = np.load('my_file.npy').item()
print(read_dictionary['hello']) 

In [4]:
import pandas as pd
a = pd.DataFrame([[1,2,3],[1,2,3]])

In [7]:
a.shape[0]

2

In [71]:
dataframe_list.to_csv('/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))
print("save to " + '/home/wuwenjun/Data/%s_%s.csv' % (time_start,time_end))

save to /home/wuwenjun/Data/20201231_20211130.csv


Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


AttributeError: 'numpy.int64' object has no attribute 'strftime'